In [1]:
from brainlit.utils import session
import napari

In [2]:
url = "s3://mouse-light-viz/precomputed_volumes/brain1"
sess = session.NeuroglancerSession(url=url, url_segments=url+"_segments", mip=0)

In [5]:
import numpy as np
from skimage import draw
import random
from cloudvolume.lib import Bbox
from io import StringIO
from csv import writer 
import pandas as pd
from tqdm import tqdm as tqdm

scale = sess.cv_segments.scales[0]["resolution"]
output = StringIO()
csv_writer = writer(output)
csv_writer.writerow(["label", "seg id", "vert id", "interp id", "data"])
sess.cv.progress = False  # don't print download
for seg_id in [2]:  # list of segments
    vertices = sess.cv_segments.skeleton.get(seg_id).vertices
    for i in tqdm(range(50)):  # len(vertices)
        v0 = (vertices[i]/scale).astype(int)  # point 0
        v1 = (vertices[i+1]/scale).astype(int)  # point 1
        coords = np.array(draw.line_nd(v0, v1))  # coords of line between points
        inds = random.sample(range(coords.shape[1]), 2)  # get X random points (here, 2)
        for ind in inds:
            coord = coords[:,ind]
            bbox = Bbox(np.subtract(coord,[3,3,3]), np.add(coord,[4,4,4]))  # get box around point
            bbox_off = Bbox(np.add(coord,[7,7,7]), np.add(coord,[14,14,14]))  # get box off point (10,10,10)
            data = sess.pull_bounds_img(bbox)  # get data
            data_off = sess.pull_bounds_img(bbox_off)
            csv_writer.writerow([1, seg_id, i, ind, data.flatten()])  # add flattened data to StringIO
            csv_writer.writerow([0, seg_id, i, ind, data_off.flatten()])
output.seek(0) # get back to the start of the BytesIO
df = pd.read_csv(output)

100%|██████████| 50/50 [03:46<00:00,  4.53s/it]


In [5]:
# with napari.gui_qt():
#     viewer = napari.view_image(img, ndisplay=3)